В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

**Задание:**

1) Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.

2) Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.

3) Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.

4) Преобразуйте категориальные переменные с помощью pd.get_dummies(). 

5) Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель?

6) Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в коэффициентах и их значимости. Какую модель лучше оставить?

7) Заполните пропуски в результатах.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
cars = pd.read_csv('cars.csv')

In [3]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
cars.shape

(205, 26)

In [5]:
cars.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [6]:
# Количество пропущенных значений
cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

**Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.**

In [9]:
# Создаем колонку с марками автомобилей
cars['company'] = cars.CarName.apply(lambda x: x.split()[0])

In [10]:
# Удаляем колонки CarName и car_ID
cars.drop(columns=['CarName', 'car_ID'], inplace=True)

In [12]:
# Количество уникальных марок
cars.company.nunique()

28

**Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!**
* 'maxda' → 'mazda'
* 'Nissan' → 'nissan'
* 'porcshce' → 'porsche'
* 'toyouta' → 'toyota'
* 'vokswagen' & 'vw' → 'volkswagen'

In [16]:
# Приводит все названия к нижнему регистру
cars.company = cars.company.apply(lambda x: x.lower())

In [37]:
def fix_company_name(name, fixed_name):
    cars.company.replace(name, fixed_name, inplace=True)

In [38]:
# Замена ошибочных названий на корректные
fix_company_name('maxda', 'mazda')
fix_company_name('porcshce', 'porsche')
fix_company_name('toyouta', 'toyota')
fix_company_name('vokswagen', 'volkswagen')
fix_company_name('vw', 'volkswagen')

In [39]:
# Количество уникальных марок
cars.company.nunique()

22

Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower' также не забыв про то, что мы предсказываем – 'price'.

**После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.**

In [40]:
ness_columns = ['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength', 'carwidth', 
                  'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']
predict_df = cars[ness_columns]

In [43]:
# Коэффициент корреляции колонки price с другими колонками
predict_df.corr()['price'].sort_values(ascending=False).round(2)

price         1.00
enginesize    0.87
curbweight    0.84
horsepower    0.81
carwidth      0.76
carlength     0.68
wheelbase     0.58
boreratio     0.55
Name: price, dtype: float64

Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies().

**Сколько колонок теперь имеется в датасете?**

In [ ]:
df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
Не забудьте присоединить к полученному датасету столбцы с переменными других типов :)